# PRELIMINARIES

In [ ]:
;;
#require "pkp"

In [ ]:
open Pkp

open Single_neuron

In [ ]:
let plot_signal x =
  let plot (module F : Figure) =
    F.barebone ();
    F.plot [| ([ x ], "w l lc 8 lw 2") |]
  in
  Juplot.draw ~fmt:`svg ~size:(800, 200) plot

In [ ]:
(* data: array of (time, state) arrays *)
let plot_voltage ?colors data =
  let open Gp in
  let plot (module F : Figure) =
    F.barebone ();
    F.set Border [ `l ];
    F.set Range (`y, (-100., 40.));
    F.set Tics (`y, `def (-100., 40., 40.)) ~o:"out nomirror";
    F.ex "set offsets graph 0.02, 0, 0, 0";
    F.set Label (`y, "V_m [mV]") ~o:"offset 4, 0 norotate";
    Array.mapi
      (fun i (t, u) ->
        let color = match colors with Some z -> z.(i) | None -> "black" in
        ( [ t; Mat.(1E3 $* col u 0) ],
          Printf.sprintf "w l lc rgb '%s' lw 2" color ) )
      data
    |> F.plot
  in
  Juplot.draw ~fmt:`svg ~size:(800, 200) plot

# HODGKIN-HUXLEY MODEL

In [ ]:
module S = HH (HH_default_prms)

## Simulation with constant input current

Here we can play with the input current and see that the neuron only spikes for large enough current:

In [ ]:
let _ =
  let input t = if t < 0.02 then 0. else 2.2E-10 in
  let t, u = S.simulate ~duration:0.1 input in
  plot_voltage [| (t, u) |]

Now, let's plot the voltage for various amplitudes of the step current: 

In [ ]:
let _ =
  [| 2.0; 2.4; 2.6; 4.0; 10.0 |]
  |> Array.map (fun c ->
         S.simulate ~duration:0.1 (fun t -> if t < 0.02 then 0. else c *. 1E-10)
     )
  |> plot_voltage
       ~colors:[| "#1b9e77"; "#d95f02"; "#7570b3"; "#e7298a"; "#66a61e" |]

Let's inspect the "gating variables": can you guess which one is which?

In [ ]:
let _ =
  let t, u =
    S.simulate ~duration:0.1 (fun t -> if t < 0.02 then 0. else 2.4E-10)
  in
  let open Gp in
  let plot (module F : Figure) =
    F.barebone ();
    F.set Range (`x, (0.015, 0.04));
    F.margins [ `l 0.3; `r 0.9 ];
    F.ex "set ytics out nomirror";
    F.set Border [ `l ];
    F.set Multiplot ();
    (* voltage at the top *)
    F.set Label (`y, "V_m [mV]") ~o:"offset 3, 0 norotate";
    F.margins [ `t 0.9; `b 0.6 ];
    F.plot [| ([ t; Mat.(1E3 $* col u 0) ], "w l lc 8 lw 2") |];
    (* gate variables *)
    F.set Label (`y, "gate");
    F.margins [ `t 0.5; `b 0.2 ];
    F.set Range (`y, (0., 1.));
    F.set Tics (`y, `def (0., 1., 1.));
    [| 1; 2; 3 |]
    |> Array.map (fun i ->
           ([ t; Mat.col u i ], Printf.sprintf "w l lc %i lw 2" i) )
    |> F.plot;
    F.unset Multiplot ()
  in
  Juplot.draw ~fmt:`svg ~size:(500, 300) plot

## Simulation with random input current

In [ ]:
let _ = plot_signal Misc.(ou_process ~tau:20E-3 ~dt:1E-3 ~duration:1.0)

In [ ]:
let noise =
  let dt = 1E-3 in
  let x = Misc.ou_process ~tau:20E-3 ~dt ~duration:1.1 in
  fun t -> Mat.get x 0 (int_of_float (t /. dt))

In [ ]:
let _ =
  let t, u = S.simulate ~duration:1.0 (fun t -> 2E-10 *. (-2. +. noise t)) in
  plot_voltage [| (t, u) |]

----

# LEAKY INTEGRATE-AND-FIRE MODEL

In [ ]:
module S = LIF (LIF_default_prms)

In [ ]:
let _ =
  [| 2.0; 2.4; 2.6; 4.0; 10.0 |]
  |> Array.map (fun c ->
         S.simulate ~duration:0.1 (fun t -> if t < 0.02 then 0. else c *. 3E-3)
     )
  |> plot_voltage
       ~colors:[| "#1b9e77"; "#d95f02"; "#7570b3"; "#e7298a"; "#66a61e" |]